<a href="https://colab.research.google.com/github/maimuna-noshin/Solar_Prediction/blob/main/solarprediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [301]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import pickle
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint
from tensorflow.keras.layers import Dense, Dropout, LSTM, Conv1D, BatchNormalization, Flatten
from sklearn.preprocessing import StandardScaler
from scipy.fftpack import fft
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

In [302]:
df=pd.read_csv("/content/updated_dataset.csv")

In [303]:
# Convert to datetime
df['date'] = pd.to_datetime(df[['Year', 'Month', 'Day', 'Hour']])




In [304]:
# Selecting features (X) and target variables (Y)
features = ['Hour', 'Temperature_A','Pressure_A', 'Day','Month', 'Surface Albedo_A', 'Cloud Type_A']
targets = ['DNI_A']  # Target variables for all three locations

In [305]:

X = df[features].values
y = df[targets].values

In [306]:
# Keep timestamps before scaling
timestamps = df[['Month', 'Day', 'Hour']]

In [307]:
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler(feature_range=(0, 1000))
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)
# Save the feature scaler
with open("scaler_X.pkl", "wb") as f:
    pickle.dump(scaler_X, f)

# Save the target scaler
with open("scaler_y.pkl", "wb") as f:
    pickle.dump(scaler_y, f)

In [308]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test, timestamps_train, timestamps_test = train_test_split(X_scaled, y_scaled,timestamps, test_size=0.2, random_state=42)
print(X_train.max())

1.0


In [309]:
model = keras.Sequential([
    keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(len(targets), activation='linear')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [310]:
# Compile the model
optimizer = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [311]:
# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=8, validation_data=(X_test, y_test))

Epoch 1/10
876/876 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 71103.1562 - mae: 197.9624 - val_loss: 34374.7305 - val_mae: 134.2465
Epoch 2/10
876/876 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 37446.2734 - mae: 128.6342 - val_loss: 29632.6172 - val_mae: 104.8848
Epoch 3/10
876/876 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 31688.2754 - mae: 109.7383 - val_loss: 28610.0391 - val_mae: 101.8249
Epoch 4/10
876/876 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 30825.0684 - mae: 106.0098 - val_loss: 28306.6348 - val_mae: 95.9818
Epoch 5/10
876/876 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 29799.0469 - mae: 103.0211 - val_loss: 26972.4492 - val_mae: 96.1106
Epoch 6/10
876/876 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 28348.5820 - mae: 99.0709 - val_loss: 26580.6172 - val_mae: 93.1278
Epoch 7/10
876/876 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 27935.4199 - mae: 98.2315 - val_loss: 26908.4492 - val_mae: 93.0658
Epoch 8/10
876/876 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 26288.4766 - mae: 94.1602 - val_loss

In [312]:
# Predict future solar irradiance for all three locations
y_pred_scaled = model.predict(X_test)
y_pred = scaler_y.inverse_transform(y_pred_scaled)
# Set negative irradiance values to 0
y_pred[y_pred < 10] = 0
# Apply nighttime condition to predicted values
nighttime_indices = (X_test[:, hour_column_index] < 6) | (X_test[:, hour_column_index] >= 18)
y_pred[nighttime_indices, :] = 0

55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step


In [313]:
# Convert predictions into a DataFrame
predictions_df = timestamps_test.copy()

# Assign each column separately
predictions_df[['DNI_A']] = y_pred

In [314]:
# Print first 100 predictions with date and hour
print(predictions_df.head(100))
predictions_df.to_csv("solar_irradiance_predictions.csv", index=False)  # Saves to a CSV]

      Month  Day  Hour  DNI_A
6056      9   10     8    0.0
5556      8   20    12    0.0
5990      9    7    14    0.0
7674     11   16    18    0.0
3319      5   19     7    0.0
...     ...  ...   ...    ...
6006      9    8     6    0.0
8063     12    2    23    0.0
14        1    1    14    0.0
5306      8   10     2    0.0
8524     12   22     4    0.0

[100 rows x 4 columns]


In [315]:
# Save using the new format:
model.save("solar_irradiance_predictions.keras")  # Or model.save("my_model") - the .keras is optional

# Load using the new format:
model = keras.models.load_model("solar_irradiance_predictions.keras")  # Or model = keras.models.load_model("my_model")

In [316]:
#  Load the scalers used during training

with open("scaler_X.pkl", "rb") as f:
    scaler_X = pickle.load(f)
with open("scaler_y.pkl", "rb") as f:
    scaler_y = pickle.load(f)

In [317]:
# Generate timestamps for the first week of March 2025 (hourly)
date_range = pd.date_range(start="2025-03-01", end="2025-03-07 23:00:00", freq="H")
future_df = pd.DataFrame({"datetime": date_range})

<ipython-input-317-0e223ba96d09>:2: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  date_range = pd.date_range(start="2025-03-01", end="2025-03-07 23:00:00", freq="H")


In [318]:
#  Feature Engineering
future_df["day"] = future_df["datetime"].dt.day
future_df["month"] = future_df["datetime"].dt.month
future_df["hour"] = future_df["datetime"].dt.hour
future_df.drop(columns=["datetime"], inplace=True)

In [319]:
# Generate random values
future_df["Temperature_A"] = np.random.uniform(27, 30, size=168)  # Temperature between 15°C to 30°C
future_df["Pressure_A"] = np.random.uniform(1000, 1050, size=168)  # Pressure between 950-1050 hPa
future_df["Surface Albedo_A"] = np.random.uniform(0.1, 0.5, size=168)  # Albedo between 0.1 - 0.5
future_df["Cloud Type_A"] = np.random.randint(0, 4, size=168)  # Cloud types (0-3)

# Merge the first week’s random values with `future_df`
#future_df = future_df.merge(future_df, on=["Month", "Day"], how="left")

# Time-based encoding (sin/cos transformation)
#future_df["hour_sin"] = np.sin(2 * np.pi * future_df["hour"] / 24)
#future_df["hour_cos"] = np.cos(2 * np.pi * future_df["hour"] / 24)

# Select the same features used in training
#X_future = future_df[['hour_sin', 'hour_cos', 'Temperature_A', 'Pressure_A', 'day', 'Month', 'Surface Albedo_A', 'Cloud Type_A']].values

In [320]:
#  Select the same features used in training
X_future = future_df[['hour', 'Temperature_A','Pressure_A', 'day','month', 'Surface Albedo_A', 'Cloud Type_A']].values
#  Scale future data using the trained scaler
X_future_scaled = scaler_X.transform(X_future)
print(X_future_scaled)


[[ 0.          0.58855756  1.25325256 ...  0.18181818  4.62964838
   0.11111111]
 [ 0.04347826  0.5779826   1.26275203 ...  0.18181818  3.99376809
   0.        ]
 [ 0.08695652  0.6618392   1.67377971 ...  0.18181818  6.58085751
   0.33333333]
 ...
 [ 0.91304348  0.58829762  1.39065896 ...  0.18181818 -0.70666455
   0.22222222]
 [ 0.95652174  0.59570665  0.72598971 ...  0.18181818  6.00336803
   0.33333333]
 [ 1.          0.65156297  1.08564861 ...  0.18181818  5.51420817
   0.        ]]


In [321]:
#  Make predictions
y_future_scaled = model.predict(X_future_scaled)

#  Inverse scale predictions
y_future = scaler_y.inverse_transform(y_future_scaled)
#  Set negative irradiance values to 0
y_future[y_future < 15] = 0

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 


In [322]:
# Save predictions to DataFrame
future_df[['DNI_A']] = y_future

#  Save to CSV
future_df.to_csv("Future_Solar_Predictions_March2025.csv", index=False)

#Display first 10 predictions
print(future_df.head(10))

   day  month  hour  Temperature_A   Pressure_A  Surface Albedo_A  \
0    1      3     0      27.386976  1028.597577          0.371482   
1    1      3     1      27.042233  1028.882561          0.339688   
2    1      3     2      29.775958  1041.213391          0.469043   
3    1      3     3      27.802040  1029.803668          0.462517   
4    1      3     4      29.048425  1041.595201          0.425336   
5    1      3     5      28.521611  1045.583853          0.123049   
6    1      3     6      27.317023  1032.052301          0.102553   
7    1      3     7      29.015670  1049.295309          0.408144   
8    1      3     8      27.611049  1035.001967          0.104695   
9    1      3     9      27.646691  1016.001883          0.386137   

   Cloud Type_A       DNI_A  
0             1    0.000000  
1             0    0.000000  
2             3    0.000000  
3             0    0.000000  
4             2   17.350122  
5             2   19.717056  
6             0    0.000000  
